In [1]:
import rasterio
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.misc import imsave, imread
from skimage.color import label2rgb
from shapely.geometry import Point

In [177]:
NDWI_correct = True

In [222]:
folders = ['austin13', 'austin14', 'austin15', 'denver13', 'denver14', 'denver15', 'dc13', 'dc14', 'dc15']

def is_urban(val) :
    return (val==1 or val==2 or val==3)

for folder in ['austin13', 'austin14'] :
    tif_file = glob.glob('./run2/data/{}/*.tif'.format(folder))[0]
    with rasterio.open(tif_file) as f :
        bounds = f.bounds
        transform = f.transform
    labels = glob.glob('./labels/*.npy')
    labels = [f for f in labels if folder in f]
    
    lats = []
    lons = []
    labels = []
    indexes = []
    
    raster = np.zeros((224*13, 224*15))
    fixed = 0
    for r in range(13) :
        for c in range(15) :
            model_labels = np.load('./run2/labels/{}_{}_{}.npy'.format(folder, r, c))
            raster[r*224:(r+1)*224, c*224:(c+1)*224] = model_labels
#             for r_i in range(0, model_labels.shape[0], 5) :
#                 for c_i in range(0, model_labels.shape[1], 6) :
#                     lon, lat = transform * (r*224 + r_i, c*224 + c_i)
#                     lons.append(lon)
#                     lats.append(lat)
                    
                    # convert to point
#                     p = Point([lon, lat])
#                     census = filter_(shapefile, sindex, p)
                    
#                     if not (census.shape[0] > 0) :
#                         continue 
            
#                     gdi = float(gdi_df[gdi_df['geoid'] == census.iloc[0]['geoid']]['gdi'])
#                     indexes.append(gdi)

                    # find index in gdi
#                     if NDWI_correct and ndwi[r*224 + r_i, c*224 + c_i] > 0 :
#                         labels.append(7)
#                         if model_labels[r_i, c_i] != 7 :
#                             fixed += 1
#                         continue 
#                     labels.append(model_labels[r_i,c_i])
    rasters.append(raster)
#     out = pd.DataFrame({'lat': pd.Series(lats), 'lon': pd.Series(lons), 'labels': pd.Series(labels), 'gdi': pd.Series(indexes)})
#     out.to_csv('run2/latlon_labels_csv/{}_labels_latlon.csv'.format(folder))
#     imsave('run2/cityjpg/{}.jpg'.format(folder), label2rgb(raster))
#     print('fixed {} false water labels'.format(fixed))

In [2]:
def get_rasters(folders) :
    rasters = []
    for folder in folders :
        labels = glob.glob('./labels/*.npy')
        labels = [f for f in labels if folder in f]

        raster = np.zeros((224*13, 224*15))
        fixed = 0
        for r in range(13) :
            for c in range(15) :
                model_labels = np.load('./run2/labels/{}_{}_{}.npy'.format(folder, r, c))
                raster[r*224:(r+1)*224, c*224:(c+1)*224] = model_labels
        rasters.append(raster)
    return rasters

In [3]:
def is_urban(val) :
    return (val==1 or val==2 or val==3)

def diff2lonlat(old, cur, transform) :
    lats = []
    lons = []
    change = []
    
    for r in range(2912) :
        for c in range(3360) :
            if not is_urban(old[r,c]) and is_urban(cur[r,c]) :
                lon, lat = transform * (r, c)
                lons.append(lon)
                lats.append(lat)
                change.append(1)
    return lons, lats, change

In [5]:
def compute_diff_city(cities=['austin13', 'austin14', 'austin15']) :
    tif_file = glob.glob('./run2/data/{}/*.tif'.format(cities[0]))[0]
    with rasterio.open(tif_file) as f :
        transform = f.transform
    dfs = []
    for i in range(len(cities)-1) :
        print('here')
        old, cur = get_rasters([cities[i], cities[i+1]])
        lons, lats, change = diff2lonlat(old, cur, transform)
        instance = pd.DataFrame({'lon':pd.Series(lons), 'lat':pd.Series(lats), 'change':pd.Series(change), 'year':i})
        dfs.append(instance)
    print(len(dfs))
    city_df = pd.concat(dfs)
    return city_df

In [6]:
df = compute_diff_city(cities=['dc13', 'dc14', 'dc15'])

here
here
2


In [7]:
df1 = df[df['year'] == 0].sample(100000)

In [9]:
df2 = df[df['year'] == 1]

In [10]:
df1.to_csv('dc14.csv')

In [11]:
df2.to_csv('dc15.csv')

In [249]:
df.to_csv('austin_change.csv')

In [250]:
df['year'].max()

1

In [251]:
df['year'].describe()

count    1.692850e+06
mean     3.880474e-01
std      4.873056e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: year, dtype: float64

In [8]:
folders = ['austin13', 'austin14', 'austin15', 'denver13', 'denver14', 'denver15', 'dc13', 'dc14', 'dc15']

def is_urban(val) :
    return (val==1 or val==2 or val==3)

for folder in ['nairobi_labels'] :
    
    raster = np.zeros((224*12, 224*12))
    fixed = 0
    for r in range(12) :
        for c in range(12) :
            model_labels = np.load('./nairobi_labels/{}_{}.npy'.format(r, c))
            raster[r*224:(r+1)*224, c*224:(c+1)*224] = model_labels
    imsave('nairobi.jpg', label2rgb(raster))

In [10]:
np.save('nairobi', raster)